In [1]:
using NBInclude
@nbinclude("functions.ipynb")

loss (generic function with 1 method)

In [ ]:
#setup
x_dim = 784
m = 200
k = 20
epoch_to_load = 20 # Load the model and test set loader
tolerance = 1e-7; max_iter = 2000; out_toggle = 0
β_list = 0:.1:1
trials = 20
opt = Flux.Optimise.ADAM(.1)


In [ ]:
_, _, decoder = load_model("MNIST", epoch_to_load)
B = randn(784, 500)/sqrt(784)
z = randn(k) # ground code vector
α_list = []
F = dct(diagm(ones(x_dim)),2)
for β in β_list
    push!(α_list, subspace_incoherence(F, β * Flux.params(decoder)[3] + (1-β)*B) )
end

In [8]:
recovery_error_matrix = zeros(trials)
recon_error_matrix = zeros(trials)
for β in β_list
    recovery_error_list = []
    recon_error_list = []
    G = Chain(
        Dense(20, 500, relu, bias = false; init =(out,in) -> Flux.params(decoder)[1]),
        Dense(500, 500, relu, bias = false; init =(out,in) -> Flux.params(decoder)[2]),
        Dense(500, 784, identity, bias = false; init =(out,in) -> β * Flux.params(decoder)[3] + (1-β)*B),
    )   #setup the generative network 


    for trials in 1:trials
        F_sub = sample_fourier(m, x_dim)  # subsampling DCT for measurement matrix
        F_sub = F_sub * sqrt(x_dim)/sqrt(m) #normalization          
        y = F_sub * G(z) # measurement vector using subsampled DCT matrix
  
        z_est = randn(k) #initialization for algorithm
        z_est = estimated_code(opt, G, y, F_sub, z_est; max_iter, tolerance, out_toggle)  # run a optimizer to solve the least squares problem

        push!(recovery_error_list, relative_error(z, z_est))    
        push!(recon_error_list, relative_error(G(z), G(z_est)))
    end

    recovery_error_matrix =  hcat(recovery_error_matrix, recovery_error_list)
    recon_error_matrix = hcat(recon_error_matrix, recon_error_list)
end




In [6]:
cs = palette([:red,  :orange, :green, :blue, :Indigo], 1000)


log_recovery_error_matrix = log10.(recovery_error_matrix[:, 2:end])
average_recov = mean(log_recovery_error_matrix, dims=1)'
std_recov = std(log_recovery_error_matrix, dims=1)'
plot(α_list, average_recov + std_recov, fillrange =  average_recov - std_recov, fillalpha = .2, c = cs[400], label = false,linecolor = invisible())
scatter!(kron(α_list',ones(trials,1))[:], log_recovery_error_matrix[:], label =:false)
p1 = plot!(α_list, average_recov, xlabel = "incoherence upper bound", ylabel = "relative recovery error", linewidth = 2, color = cs[1000], label =false)
yticks = [-6:2:0;]
yticks!(yticks, [L"10^{%$y}" for y in yticks])



log_recon_error_matrix = log10.(recon_error_matrix[:, 2:end])
average_recon = mean(log_recon_error_matrix, dims=1)'
std_recon = std(log_recon_error_matrix, dims=1)'
plot(α_list, average_recon + std_recon, fillrange = average_recon - std_recon, fillalpha = .2, c = cs[400], label = false, linecolor = invisible())
scatter!(kron(α_list',ones(trials,1))[:], log_recon_error_matrix[:], label =:false)
p2 = plot!(α_list, average_recon, xlabel = "incoherence upper bound", ylabel = "relative reconstruction error", linewidth = 2, color = cs[1000], label =false )
yticks = [-8:2:0;]
yticks!(yticks, [L"10^{%$y}" for y in yticks])
plot(p1,p2,layout = 2)

11-element Vector{Any}:
 0.8467596642220849
 0.9311394555706298
 0.9646827042585921
 0.9774385492232273
 0.983521608262825
 0.9859371517903763
 0.9865932104842539
 0.986459116125491
 0.9853124576955354
 0.9829007370386266
 0.9787944574668505

In [7]:
savefig("figures/Full_MNIST_error.pdf")
save("save_data/Full_MNIST_error.jld", "recovery error", recovery_error_matrix, "reconstruction error", recon_error_matrix, "α_list", α_list)

0.7985957062499249